In [1]:
import itertools

def vcg_auction_multiple_items(bids, items):
    """
    Implements a VCG auction for multiple items with multiple bidders.
    
    Args:
    bids (dict): A dictionary where keys are bidder names and values are lists of tuples.
                 Each tuple contains a combination of items and the bid for that combination.
    items (list): A list of all available items.
    
    Returns:
    dict: A dictionary containing the winners, their allocated items, and the prices they pay.
    """
    # Generate all possible allocations of items
    # This function generates all possible ways to distribute 'n' items among 'n' bidders.
    # It uses the itertools.product to create all possible partitions of the items list.
    # Each partition is a tuple where the value at each position indicates which bidder gets the item.
    # Example: For items ['item1', 'item2'], the partitions could be [(0, 0), (0, 1), (1, 0), (1, 1)]
    # where (0, 1) means item1 goes to bidder 0 and item2 goes to bidder 1
    #Formula: The number of possible allocations is (n+1)^n where n is the number of items.
    def all_possible_allocations(items):
        n = len(items)
        all_allocations = []
        for partition in itertools.product(range(n+1), repeat=n):
            parts = [[] for _ in range(n)]
            for item, part in zip(items, partition):
                if part < n:
                    parts[part].append(item)
            all_allocations.append(parts)
        return all_allocations

    all_allocations = all_possible_allocations(items)
    
    # Calculate the total value for each allocation
    # This function calculates the total value of an allocation.
    # For each bidder i, it checks their allocated items part.
    # It adds the highest bid value for a combination of items in part.
    # The total value is the sum of all highest bids for the allocation.
    # Formula: total_value(allocation) = Σ bid for each part
    def total_value(allocation):
        value = 0
        for i, part in enumerate(allocation):
            for combo, bid in bids.get(i, []):
                if set(combo) <= set(part):
                    value += bid
                    break
        return value
    
    # Find the optimal allocation
    # This finds the allocation that maximizes the total value.
    # It uses the max function with the total_value function as the key.
    # The optimal_value is the highest total value obtained.
    #Formula: optimal_allocation = argmax_allocation(total_value(allocation))
    optimal_allocation = max(all_allocations, key=total_value)
    optimal_value = total_value(optimal_allocation)
    
    # Determine the payments
    # For each bidder i, this calculates the payment they need to make.
    # It first creates all possible allocations without the items assigned to bidder i.
    # The value_without_i is the highest value obtained without bidder i's items.
    # The payment for bidder i is the optimal_value minus value_without_i.
    #Formula: payment_i = optimal_value - value_without_i
    payments = {}
    for i, part in enumerate(optimal_allocation):
        if part:
            without_i = all_possible_allocations([item for j, items in enumerate(optimal_allocation) if j != i for item in items])
            value_without_i = max(total_value(allocation) for allocation in without_i)
            payments[i] = optimal_value - value_without_i
    
    return {
        'allocation': optimal_allocation,
        'payments': payments
    }

# Example usage
bids = {
    'Alice': [(('item1',), 100), (('item1', 'item2'), 150)],
    'Bob': [(('item2',), 80), (('item1', 'item2'), 140)],
    'Charlie': [(('item1',), 90), (('item2',), 85)]
}

items = ['item1', 'item2']
result = vcg_auction_multiple_items(bids, items)

print("Allocation:", result['allocation'])
print("Payments:", result['payments'])

Allocation: [['item1', 'item2'], []]
Payments: {0: 0}
